In [2]:
!pip install torch==2.3.0 transformers==4.28.0 datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import transformers
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

In [4]:
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(examples["text"], truncation=True,
                     padding="max_length", max_length=max_length)

In [5]:
def main():
  model_name = "distilbert-base-uncased"
  tokenizer = DistilBertTokenizer.from_pretrained(model_name)
  model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)

  dataset = load_dataset("ag_news",split="train[:2000]")
  dataset = dataset.train_test_split(test_size=0.1)
  train_dataset = dataset["train"]
  test_dataset = dataset["test"]

  tokenized_train =train_dataset.map(
      lambda x:preprocess_function(x,tokenizer),
      batched=True,
      remove_columns = ["text"]
  )
  tokenized_test = test_dataset.map(
      lambda x:preprocess_function(x,tokenizer),
      batched=True,
      remove_columns = ["text"]
  )

  accuracy = evaluate.load("accuracy")
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)\

  training_args = TrainingArguments(
      output_dir = './distilbert_topics',
      evaluation_strategy = "epoch",
      learning_rate = 2e-5,
      per_device_train_batch_size = 8,
      per_device_eval_batch_size = 8,
      num_train_epochs = 2,
      save_total_limit = 2,
      fp16 = torch.cuda.is_available(),
      logging_steps =100,
      report_to = "none"
  )

  trainer = Trainer(
      model = model,
      args = training_args,
      train_dataset = tokenized_train,
      eval_dataset = tokenized_test,
      tokenizer = tokenizer,
      compute_metrics = compute_metrics
  )

  trainer.train()
  trainer.save_model("./distilbert_topics")
  tokenizer.save_pretrained("./distilbert_topics")

  # Inferece step
  def predict_topic(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt",max_length=128, truncation=True, padding=True)
    model.eval()
    with torch.no_grad():
      outputs = model(**inputs.to(model.device))

    prediction = torch.argmax(outputs.logits, dim=1).item()
    topics = {0:"World",1:"Sports",2:"Business",3:"Sci/Tech"}
    return topics[prediction]

  sample_texts = [
        "UN holds emergency meeting to address rising tensions in the Middle East.",
        "Tech giant announces $2 billion investment in renewable energy projects.",
        "Local high school team wins state championship in a thrilling overtime match.",
        "Scientists develop new battery technology that charges in seconds."
    ]

  for text in sample_texts:
    topic = predict_topic(text, model, tokenizer)
    print(f"Text: {text}\nPredicted Topic: {topic}\n")

if __name__ == "__main__":
  main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.449500,0.416278,0.865000
2,0.289600,0.389691,0.875000


Text: UN holds emergency meeting to address rising tensions in the Middle East.
Predicted Topic: World

Text: Tech giant announces $2 billion investment in renewable energy projects.
Predicted Topic: Business

Text: Local high school team wins state championship in a thrilling overtime match.
Predicted Topic: Sports

Text: Scientists develop new battery technology that charges in seconds.
Predicted Topic: Sci/Tech

